# Taxi Trips in NYC Data Cleaning

This notebook contains all the code needed to clean the raw data from taxi_tripdata.csv and taxi_zones.csv to create the new, cleaned dataset taxi_clean.csv

## Imports and Grabbing Data

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
dtypes = {'VendorID': 'Int64', 'store_and_fwd_flag': 'str', 'RatecodeID': 'Int64', 'passenger_count': 'Int64', 'payment_type': 'Int64', 'trip_type': 'Int64'}
parse_dates = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']
df = pd.read_csv('data/taxi_tripdata.csv', dtype=dtypes, parse_dates=parse_dates)

In [3]:
# First few rows of the raw data:
df.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,1,2021-07-01 00:30:52,2021-07-01 00:35:36,N,1,74,168,1,1.20,6.0,0.50,0.5,0.00,0.0,NaN,0.3,7.30,2,1,0.00
1,2,2021-07-01 00:25:36,2021-07-01 01:01:31,N,1,116,265,2,13.69,42.0,0.50,0.5,0.00,0.0,NaN,0.3,43.30,2,1,0.00
2,2,2021-07-01 00:05:58,2021-07-01 00:12:00,N,1,97,33,1,0.95,6.5,0.50,0.5,2.34,0.0,NaN,0.3,10.14,1,1,0.00
3,2,2021-07-01 00:41:40,2021-07-01 00:47:23,N,1,74,42,1,1.24,6.5,0.50,0.5,0.00,0.0,NaN,0.3,7.80,2,1,0.00
4,2,2021-07-01 00:51:32,2021-07-01 00:58:46,N,1,42,244,1,1.10,7.0,0.50,0.5,0.00,0.0,NaN,0.3,8.30,2,1,0.00
5,1,2021-07-01 00:05:00,2021-07-01 00:11:50,N,1,24,239,1,1.90,8.0,3.25,0.5,3.00,0.0,NaN,0.3,15.05,1,1,2.75
6,2,2021-07-01 00:57:14,2021-07-01 01:27:43,N,1,75,243,1,0.00,17.5,0.50,0.5,0.00,0.0,NaN,0.3,18.80,2,1,0.00
7,2,2021-07-01 00:27:36,2021-07-01 00:32:35,N,1,82,82,1,0.66,5.0,0.50,0.5,0.00,0.0,NaN,0.3,6.30,2,1,0.00
8,2,2021-07-01 00:29:09,2021-07-01 00:34:18,N,1,74,42,1,1.72,7.0,0.50,0.5,2.08,0.0,NaN,0.3,10.38,1,1,0.00
9,2,2021-07-01 00:41:33,2021-07-01 00:49:24,N,1,41,42,1,1.37,7.5,0.50,0.5,0.00,0.0,NaN,0.3,8.80,2,1,0.00


## Dropping Unnecessary Columns
Removing the `ehail_fee` column.

In [4]:
df_drops = df.drop('ehail_fee', axis=1)
df_drops.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,1,2021-07-01 00:30:52,2021-07-01 00:35:36,N,1,74,168,1,1.20,6.0,0.50,0.5,0.00,0.0,0.3,7.30,2,1,0.00
1,2,2021-07-01 00:25:36,2021-07-01 01:01:31,N,1,116,265,2,13.69,42.0,0.50,0.5,0.00,0.0,0.3,43.30,2,1,0.00
2,2,2021-07-01 00:05:58,2021-07-01 00:12:00,N,1,97,33,1,0.95,6.5,0.50,0.5,2.34,0.0,0.3,10.14,1,1,0.00
3,2,2021-07-01 00:41:40,2021-07-01 00:47:23,N,1,74,42,1,1.24,6.5,0.50,0.5,0.00,0.0,0.3,7.80,2,1,0.00
4,2,2021-07-01 00:51:32,2021-07-01 00:58:46,N,1,42,244,1,1.10,7.0,0.50,0.5,0.00,0.0,0.3,8.30,2,1,0.00
5,1,2021-07-01 00:05:00,2021-07-01 00:11:50,N,1,24,239,1,1.90,8.0,3.25,0.5,3.00,0.0,0.3,15.05,1,1,2.75
6,2,2021-07-01 00:57:14,2021-07-01 01:27:43,N,1,75,243,1,0.00,17.5,0.50,0.5,0.00,0.0,0.3,18.80,2,1,0.00
7,2,2021-07-01 00:27:36,2021-07-01 00:32:35,N,1,82,82,1,0.66,5.0,0.50,0.5,0.00,0.0,0.3,6.30,2,1,0.00
8,2,2021-07-01 00:29:09,2021-07-01 00:34:18,N,1,74,42,1,1.72,7.0,0.50,0.5,2.08,0.0,0.3,10.38,1,1,0.00
9,2,2021-07-01 00:41:33,2021-07-01 00:49:24,N,1,41,42,1,1.37,7.5,0.50,0.5,0.00,0.0,0.3,8.80,2,1,0.00


## Remove Invalid Rows

Remove rows where the `trip_distance` is 0, or the `tip_amount` is negative. Because the majority of the data involves trips in July, we decided to remove any outlier data from other months.

In [5]:
print('Remove ' + str(len(df_drops[df_drops['trip_distance'] == 0])) + ' rows with a trip_distance of 0')
df_rev1 = df_drops[df_drops['trip_distance'] > 0]

Remove 3455 rows with a trip_distance of 0


In [6]:
print('Remove ' + str(len(df_rev1[df_rev1['tip_amount'] < 0])) + ' row(s) with a negative tip')
df_rev2 = df_rev1[df_rev1['tip_amount'] >= 0]

Remove 1 row(s) with a negative tip


In [7]:
print('Remove ' + str(len(df_rev2[df_rev2['lpep_pickup_datetime'].dt.month != 7])) + ' rows for trips not in July')
df_rev3 = df_rev2[df_rev2['lpep_pickup_datetime'].dt.month == 7]

Remove 16 rows for trips not in July


In [8]:
print('Remove ' + str(len(df_rev3[df_rev3['fare_amount'] <= 0])) + ' rows with a negative fare')
df_rev4 = df_rev3[df_rev3['fare_amount'] > 0]

Remove 249 rows with a negative fare


## Adding New Columns

We will begin by adding the following columns to make it easier to analyze the times:
- day of the week for the trip (0 = Monday...6 = Sunday)
- day of the month for the trip
- time of the day for the trip

In [9]:
df_add = df_rev4.copy()
df_add['day_of_week'] = df_add['lpep_pickup_datetime'].dt.dayofweek
df_add['day_of_month'] = df_add['lpep_pickup_datetime'].dt.day
df_add['hour_of_day'] = df_add['lpep_pickup_datetime'].dt.hour
df_add['trip_duration'] = round((df_add['lpep_dropoff_datetime'] - df_add['lpep_pickup_datetime']).dt.total_seconds())
df_add[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'day_of_week', 'day_of_month', 'hour_of_day', 'trip_duration']]

,lpep_pickup_datetime,lpep_dropoff_datetime,day_of_week,day_of_month,hour_of_day,trip_duration
0,2021-07-01 00:30:52,2021-07-01 00:35:36,3,1,0,284.0
1,2021-07-01 00:25:36,2021-07-01 01:01:31,3,1,0,2155.0
2,2021-07-01 00:05:58,2021-07-01 00:12:00,3,1,0,362.0
3,2021-07-01 00:41:40,2021-07-01 00:47:23,3,1,0,343.0
4,2021-07-01 00:51:32,2021-07-01 00:58:46,3,1,0,434.0
...,...,...,...,...,...,...
83686,2021-07-02 07:59:00,2021-07-02 08:33:00,4,2,7,2040.0
83687,2021-07-02 07:02:00,2021-07-02 07:18:00,4,2,7,960.0
83688,2021-07-02 07:53:00,2021-07-02 08:15:00,4,2,7,1320.0
83689,2021-07-02 07:58:00,2021-07-02 08:30:00,4,2,7,1920.0


Then, we'll add a column for the total fare without the tip (`total_amount` - `tip_amount`) and the fare per distance ($/mile not including the tip amount).

In [10]:
df_add['total_without_tip'] = round(df_add['total_amount'] - df_add['tip_amount'], 2)
df_add['fare_per_mile'] = round(df_add['total_without_tip'] / df_add['trip_distance'], 2)
df_add[['tip_amount', 'total_amount', 'total_without_tip', 'trip_distance', 'fare_per_mile']]

,tip_amount,total_amount,total_without_tip,trip_distance,fare_per_mile
0,0.00,7.30,7.30,1.20,6.08
1,0.00,43.30,43.30,13.69,3.16
2,2.34,10.14,7.80,0.95,8.21
3,0.00,7.80,7.80,1.24,6.29
4,0.00,8.30,8.30,1.10,7.55
...,...,...,...,...,...
83686,0.00,59.84,59.84,18.04,3.32
83687,3.66,25.87,22.21,5.56,3.99
83688,0.00,22.75,22.75,5.13,4.43
83689,0.00,54.12,54.12,12.58,4.30


## Location Data

In [11]:
taxi_zones = pd.read_csv('data/taxi_zones.csv')
taxi_zones.head()

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island


In [12]:
taxi_zones_less = taxi_zones[['the_geom', 'LocationID', 'zone', 'borough']]
pickup_dict = {'the_geom':'PUGeom', 'zone': 'PUZone', 'borough': 'PUBorough'}
dropoff_dict = {'the_geom':'DOGeom', 'zone': 'DOZone', 'borough': 'DOBorough'}
df_pickups = df_add.merge(right=taxi_zones_less, left_on='PULocationID', right_on='LocationID').drop(
    'LocationID', axis=1).rename(pickup_dict, axis='columns')
df_all_geoms = df_pickups.merge(right=taxi_zones_less, left_on='DOLocationID', right_on='LocationID').drop(
    'LocationID', axis=1).rename(dropoff_dict, axis='columns')
df_all_geoms

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,hour_of_day,trip_duration,total_without_tip,fare_per_mile,PUGeom,PUZone,PUBorough,DOGeom,DOZone,DOBorough
0,1,2021-07-01 00:30:52,2021-07-01 00:35:36,N,1,74,168,1,1.20,6.00,...,0,284.0,7.30,6.08,MULTIPOLYGON (((-73.93380589881859 40.81651249...,East Harlem North,Manhattan,MULTIPOLYGON (((-73.92480953899991 40.81565901...,Mott Haven/Port Morris,Bronx
1,2,2021-07-01 07:44:32,2021-07-01 07:50:42,N,1,74,168,1,1.57,7.00,...,7,370.0,7.80,4.97,MULTIPOLYGON (((-73.93380589881859 40.81651249...,East Harlem North,Manhattan,MULTIPOLYGON (((-73.92480953899991 40.81565901...,Mott Haven/Port Morris,Bronx
2,2,2021-07-01 17:37:32,2021-07-01 18:00:45,N,1,74,168,1,1.31,14.00,...,17,1393.0,15.80,12.06,MULTIPOLYGON (((-73.93380589881859 40.81651249...,East Harlem North,Manhattan,MULTIPOLYGON (((-73.92480953899991 40.81565901...,Mott Haven/Port Morris,Bronx
3,2,2021-07-02 06:33:23,2021-07-02 06:38:04,N,1,74,168,1,1.97,7.50,...,6,281.0,8.30,4.21,MULTIPOLYGON (((-73.93380589881859 40.81651249...,East Harlem North,Manhattan,MULTIPOLYGON (((-73.92480953899991 40.81565901...,Mott Haven/Port Morris,Bronx
4,2,2021-07-02 18:48:40,2021-07-02 18:56:17,N,1,74,168,1,1.43,7.00,...,18,457.0,8.80,6.15,MULTIPOLYGON (((-73.93380589881859 40.81651249...,East Harlem North,Manhattan,MULTIPOLYGON (((-73.92480953899991 40.81565901...,Mott Haven/Port Morris,Bronx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79847,2,2021-07-27 19:28:52,2021-07-27 20:10:02,N,1,14,6,1,28.90,76.50,...,19,2470.0,84.85,2.94,MULTIPOLYGON (((-74.03407329297129 40.64431393...,Bay Ridge,Brooklyn,MULTIPOLYGON (((-74.06367318899999 40.60219816...,Arrochar/Fort Wadsworth,Staten Island
79848,2,2021-07-29 19:23:15,2021-07-29 20:19:28,N,1,54,6,1,26.53,75.50,...,19,3373.0,83.85,3.16,MULTIPOLYGON (((-74.00174362072502 40.69240674...,Columbia Street,Brooklyn,MULTIPOLYGON (((-74.06367318899999 40.60219816...,Arrochar/Fort Wadsworth,Staten Island
79849,2,2021-07-07 15:43:32,2021-07-07 17:28:21,N,1,14,99,1,15.62,77.50,...,15,6289.0,84.85,5.43,MULTIPOLYGON (((-74.03407329297129 40.64431393...,Bay Ridge,Brooklyn,MULTIPOLYGON (((-74.16842916199994 40.58649792...,Freshkills Park,Staten Island
79850,2,2021-07-27 22:50:07,2021-07-27 22:56:26,N,1,5,99,1,1.58,7.50,...,22,379.0,8.80,5.57,MULTIPOLYGON (((-74.17421738099989 40.56256808...,Arden Heights,Staten Island,MULTIPOLYGON (((-74.16842916199994 40.58649792...,Freshkills Park,Staten Island


## Save Data

In [13]:
df_add.to_csv('data/taxi_clean.csv', index=False)